In [1]:
!pip install "sagemaker>=2.48.0" "transformers==4.6.1" "datasets[s3]==1.6.2" --upgrade

     |████████████████████████████████| 522 kB 6.9 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 2.2 MB 33.9 MB/s            
     |████████████████████████████████| 221 kB 47.4 MB/s            
     |████████████████████████████████| 3.3 MB 72.3 MB/s            
     |████████████████████████████████| 895 kB 54.8 MB/s            
     |████████████████████████████████| 69 kB 11.2 MB/s            
     |████████████████████████████████| 211 kB 45.2 MB/s            
     |████████████████████████████████| 130 kB 45.1 MB/s            
     |████████████████████████████████| 7.2 MB 38.9 MB/s            
     |████████████████████████████████| 73 kB 554 kB/s             
  Using cached sagemaker-2.86.2-py2.py3-none-any.whl
     |████████████████████████████████| 521 kB 79.6 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 521 kB 75.8 MB/s            
  Preparing metadata (setup.py) ... done

In [2]:
import sagemaker.huggingface
import sagemaker

sess = sagemaker.Session()
role = sagemaker.get_execution_role()

print(f"IAM role arn used for running training: {role}")
print(f"S3 bucket used for storing artifacts: {sess.default_bucket()}")

IAM role arn used for running training: arn:aws:iam::847380964353:role/spot-bot-SpotSageMakerExecutionRole-TP8BLT3Z5JJL
S3 bucket used for storing artifacts: sagemaker-us-west-2-847380964353


## train-classification

In [17]:
## prepare data
import pandas as pd
import os

test = pd.read_csv(os.path.join('./data/shulex/bert-pair', "test_NLI_M.csv"),header=None,sep="\t")
train = pd.read_csv(os.path.join('./data/shulex/bert-pair', "train_NLI_M.csv"),header=None,sep="\t")

test.columns=["idx","label","text"]

train.columns=["idx","label","text"]


train[["label","text"]].to_csv('./data/shulex/bert-pair/train.csv',index=False,encoding='utf-8')
test[["label","text"]].to_csv('./data/shulex/bert-pair/test.csv',index=False,encoding='utf-8')


In [18]:
test.head()

,idx,label,text
0,5380,none,what do you think of the easy_assembly of it ?...
1,5380,none,what do you think of the would_recommend of it...
2,5380,none,what do you think of the will_repurchase of it...
3,5380,none,what do you think of the light_weight of it ? ...
4,5380,none,what do you think of the worth_the_price of it...


In [19]:
import boto3
prefix='hp-datalab'

bucket = sess.default_bucket() 
boto3.Session().resource("s3").Bucket(bucket).Object(
    os.path.join(prefix, "train/train.csv")
).upload_file("./data/shulex/bert-pair/train.csv")
boto3.Session().resource("s3").Bucket(bucket).Object(
    os.path.join(prefix, "test/test.csv")
).upload_file("./data/shulex/bert-pair/test.csv")

In [20]:
training_input_path = f's3://{sess.default_bucket()}/{prefix}/train/train.csv'
test_input_path = f's3://{sess.default_bucket()}/{prefix}/test/test.csv'

In [21]:
git_config = {'repo': 'https://github.com/huggingface/transformers.git','branch': 'v4.6.1'} # v4.6.1 is referring to the `transformers_version` you use in the estimator.

In [22]:
hyperparameters={'per_device_train_batch_size':4,
                 'per_device_eval_batch_size': 4,
                 'model_name_or_path': 'roberta-large',
                 'train_file':'/opt/ml/input/data/train/train.csv',
                 'validation_file':'/opt/ml/input/data/test/test.csv',
                 'test_file':'/opt/ml/input/data/test/test.csv',
                 'do_train': True,
                 'do_predict': True,
                 'do_eval': True,
                 'save_total_limit':3,
                 'num_train_epochs': 3,
                 'output_dir': '/opt/ml/model',
                 'num_train_epochs': 1,
                 'learning_rate': 5e-5,
                 'seed': 7,
                 'fp16': False,
                 'eval_steps': 1000,
                 }


In [23]:
from sagemaker.huggingface import HuggingFace

# create the Estimator
huggingface_estimator = HuggingFace(
      entry_point='run_glue.py', # script
      source_dir='./examples/pytorch/text-classification', # relative path to example
      git_config=git_config,
      instance_type='ml.p3.8xlarge',
      instance_count=1,
      volume_size=500,
      transformers_version='4.6',
      pytorch_version='1.7',
      py_version='py36',
      role=role,
      base_job_name='roberta-large-epoch3',
      hyperparameters = hyperparameters
)


In [ ]:
huggingface_estimator.fit({'train':training_input_path,'test':test_input_path})


2022-04-25 08:53:42 Starting - Starting the training job...ProfilerReport-1650876815: InProgress
...
2022-04-25 08:54:28 Starting - Preparing the instances for training......
2022-04-25 08:55:43 Downloading - Downloading input data...
2022-04-25 08:56:06 Training - Downloading the training image..................
2022-04-25 08:59:07 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-04-25 08:59:07,758 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-04-25 08:59:07,801 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-04-25 08:59:07,808 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-04-25 08:59:08,138 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.6 -m pip install -r requir

## test accuracy

## deploy

In [ ]:
from sagemaker.huggingface import HuggingFaceModel
import sagemaker

role = sagemaker.get_execution_role()

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   model_data="s3://sagemaker-us-west-2-847380964353/xlm-roberta-base-epoch1-2021-11-08-08-17-03-658/output/model.tar.gz",  # path to your trained sagemaker model
   role=role, # iam role with permissions to create an Endpoint
   transformers_version="4.6", # transformers version used
   pytorch_version="1.7", # pytorch version used
   py_version="py36", # python version of the DLC
)

In [ ]:
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type="ml.g4dn.xlarge"
)

In [ ]:
# example request, you always need to define "inputs"
data = {
   "inputs": "The new Hugging Face SageMaker DLC makes it super easy to deploy models in production. I love it!"
}

# request
predictor.predict(data)